In [21]:
import pandas as pd

In [22]:
mu_partial_agonist = pd.read_csv("mu_partial_agonist.csv")
titles = mu_partial_agonist.aiddesc.drop_duplicates().values.tolist()
# for title in titles:
#     print(title[:70])
len(titles)

45

In [23]:
mu_partial_agonist["links"] = mu_partial_agonist.aid.apply(lambda x : "https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{}/CSV".format(x))
mu_partial_agonist["links"]

0      https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
1      https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
2      https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
3      https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
4      https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
                             ...                        
140    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
141    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
142    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
143    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
144    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
Name: links, Length: 145, dtype: object

In [24]:
idx = 18
current_title = titles[idx]
current_title

"Title: Redefining the structure-activity relationships of 2,6-methano-3-benzazocines. Part 8. High affinity ligands for opioid receptors in the picomolar Ki range: oxygenated N-(2-[1,1'-biphenyl]-4-ylethyl) analogues of 8-CAC._||_Abstract: N-[2-(4'-methoxy[1,1'-biphenyl]-4-yl)ethyl]-8-CAC (1) is a high affinity (K(i)=0.084 nM) ligand for the μ opioid receptor and served as the lead compound for this study. Analogues of 1 were made in hopes of identifying an SAR within a series of oxygenated (distal) phenyl derivatives. A number of new analogues were made having single-digit pM affinity for the μ receptor. The most potent was the 3',4'-methylenedioxy analogue 18 (K(i)=1.6 pM)."

In [25]:
aid_df = mu_partial_agonist[mu_partial_agonist.aiddesc == current_title]
aid_links = aid_df.links.to_list()
aid_links

['https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/719297/CSV',
 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/719298/CSV']

In [26]:
bioassays_df = []
for i, link in enumerate(aid_links):
    df = pd.read_csv(link)
    filtered_df = df[df.PUBCHEM_RESULT_TAG.str.isnumeric()]
    # filtered_df = filtered_df[  ['PUBCHEM_CID', 'Standard Type', 'Standard Relation', 'Standard Value', 'Standard Units'] ]
    try:
        filtered_df = filtered_df[  ['PUBCHEM_CID', 'Standard Type','Standard Value', 'Standard Units'] ]
        filtered_df.PUBCHEM_CID = filtered_df.PUBCHEM_CID.astype('int')
        # filtered_df.set_index('PUBCHEM_CID', inplace=True)
        filtered_df.reset_index(drop=True, inplace=True)
        filtered_df.rename(columns = {'Standard Type':'Standard_Type_' + str(i), 
                                    'Standard Value':'Standard_Value_' + str(i), 
                                    'Standard Units': 'Standard_Units_' + str(i)}, inplace = True)
        bioassays_df.append(filtered_df)
    except:
        print("idx ",i , " not working")
        pass

# bioassays_df

In [27]:
df = bioassays_df[0]
for mined_df in bioassays_df[1:]:
    df = df.merge(mined_df, on='PUBCHEM_CID', how='outer')
df

,PUBCHEM_CID,Standard_Type_0,Standard_Value_0,Standard_Units_0,Standard_Type_1,Standard_Value_1,Standard_Units_1
0,58443217,Emax,21,%,EC50,0.6,nM
1,71454361,Emax,31,%,EC50,0.5,nM
2,71454362,Emax,32,%,EC50,1.6,nM
3,71456106,Emax,48,%,EC50,1.5,nM
4,24873473,Emax,25,%,EC50,0.22,nM
5,58443234,Emax,29,%,EC50,0.15,nM
6,58443192,Emax,35,%,EC50,2.3,nM


In [28]:
csv_name = current_title
print(csv_name)
df.to_csv("{}_{}.csv".format(csv_name[6:30], idx))

Title: Redefining the structure-activity relationships of 2,6-methano-3-benzazocines. Part 8. High affinity ligands for opioid receptors in the picomolar Ki range: oxygenated N-(2-[1,1'-biphenyl]-4-ylethyl) analogues of 8-CAC._||_Abstract: N-[2-(4'-methoxy[1,1'-biphenyl]-4-yl)ethyl]-8-CAC (1) is a high affinity (K(i)=0.084 nM) ligand for the μ opioid receptor and served as the lead compound for this study. Analogues of 1 were made in hopes of identifying an SAR within a series of oxygenated (distal) phenyl derivatives. A number of new analogues were made having single-digit pM affinity for the μ receptor. The most potent was the 3',4'-methylenedioxy analogue 18 (K(i)=1.6 pM).


In [29]:
# df = pd.read_csv("https://pubchem.ncbi.nlm.nih.gov/bioassay/395302#section=Data-Table")
# df = pd.read_csv("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/395302/CSV")
# df

In [30]:
# filtered_df = df[df.PUBCHEM_RESULT_TAG.str.isnumeric()]
# filtered_df = filtered_df[  ['PUBCHEM_CID', 'Standard Type', 'Standard Relation' ,'Standard Value', 'Standard Units'] ]
# filtered_df.PUBCHEM_CID = filtered_df.PUBCHEM_CID.astype('int')
# filtered_df.set_index('PUBCHEM_CID', inplace=True)
# filtered_df